In [ ]:
%pyspark
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
music_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Music_v1_00.tsv.gz"), sep="\t", header=True)
music_df.show()

In [ ]:
%pyspark
# Row Count
music_df.count()

In [ ]:
%pyspark
from pyspark.sql.functions import to_date
# Review DataFrame
review_id_df = music_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

In [ ]:
%pyspark
products_df = music_df.select(["product_id", "product_title"]).drop_duplicates()
products_df.show(10)

In [ ]:
%pyspark
reviews_df = music_df.select(["review_id", "review_headline", "review_body"])
reviews_df.show(10)

In [ ]:
%pyspark
customers_df = music_df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

In [ ]:
%pyspark
vine_df = music_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show(10)

In [ ]:
%pyspark
# Connect to DB
# Plug in your values for endpoint and password
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/postgres"
config = {"user":"postgres", "<password>": "drowssap", "driver":"org.postgresql.Driver"}

In [ ]:
%pyspark
# Write review_id_df to table in RDS
reviews_df.write.jdbc(url=jdbc_url, table='reviews', mode=mode, properties=config)

In [ ]:
%pyspark
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
%pyspark
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
%pyspark
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
%pyspark
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vines', mode=mode, properties=config)